In [347]:
import pybaseball
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

pd.set_option('display.max_rows', 1600)
pd.set_option('display.max_columns',1500)
pd.set_option('display.width', 1500)

In [348]:
injuries2 = pd.read_excel("/Users/nchapman/Documents/Documents/MSAS/spring_2020/MAT8406/Pitcher Injury Project/Injuries_Full.xlsx")

In [349]:
injuries2['Player'] =injuries2['Player'].str.upper()

## Tommy John Injuries

In [350]:
tj_injuries = pd.read_excel("Tommy John Surgery List (@MLBPlayerAnalys).xlsx", dtype='object')
mlb_injuries = tj_injuries[(tj_injuries['Level'] == 'MLB') & (tj_injuries['Position'] == 'P')]


In [351]:
mlb_injuries['Player'] = mlb_injuries['Player'].str.replace("Mark Leiter Jr.",'Mark Leiter')
mlb_injuries['Player'] = mlb_injuries['Player'].str.replace("Manuel Corpas",'MANNY CORPAS')


/Users/nchapman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/nchapman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [352]:
mlb_injuries['Player'] = mlb_injuries['Player'].str.strip()

/Users/nchapman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [353]:
mlb_injuries['TJ Surgery Date'] = pd.to_datetime(mlb_injuries['TJ Surgery Date'])

/Users/nchapman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [354]:
mlb_injuries['surgery_year'] = [x.year - 1 if
                                x.month < 6 else
                                x.year for x in mlb_injuries['TJ Surgery Date']]

/Users/nchapman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [355]:
mlb_injuries['Player'] = mlb_injuries['Player'].str.upper()

/Users/nchapman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [356]:
mlb_injuries = mlb_injuries[['Player','surgery_year']]

In [357]:
mlb_injuries['tj'] = 1

In [358]:
def tj_injury(player, season):
    if mlb_injuries[(mlb_injuries['Player'] == player) & 
                   (mlb_injuries['surgery_year'] < season)]['tj'].sum() > 0:
        return 1
    else:
        return 0 

In [359]:
def tj_injury_year(player, season):
    
    temp = mlb_injuries[(mlb_injuries['Player'] == player) & 
                   (mlb_injuries['surgery_year'] < season)]
    
    if temp['tj'].sum() > 0:
        return temp["surgery_year"].max()
    else:
        return 0 

## Grab Pitcher Stats By Year

In [360]:
#pitching_stats = pybaseball.pitching_stats(2000, 2020)
pitching_stats = pd.read_csv("/Users/nchapman/Documents/Documents/MSAS/spring_2020/MAT8406/Pitcher Injury Project/pitching_stats_2009_2020.csv")

In [361]:
pitching_stats.to_csv("/Users/nchapman/Documents/Documents/MSAS/spring_2020/MAT8406/Pitcher Injury Project/pitching_stats_2009_2020.csv",
                     index=False)

In [362]:
pitch_types = ['FB%', 'SL%', 'CT%', 'CB%', 'SF%', 'KN%']

In [363]:
pitching_stats.sort_values("Season", inplace=True)

In [364]:
pitching_stats['Name'] = pitching_stats['Name'].str.upper()

In [365]:
pitchers = pitching_stats.groupby("Name").agg({"Season":"min"}).reset_index()

pitchers_gt_2001 = pitchers[pitchers['Season'] >= 2001]

In [366]:
pitches_injury = pd.merge(pitching_stats, injuries2,
         left_on =['Name','Season'],
         right_on = ['Player','Season'],
        how='left')

In [367]:
ps = pitches_injury.groupby("Name").agg({"Season":pd.Series.nunique,
                                        "Age":"min"})

In [368]:
ps = ps.rename(columns={"Season":"Num_Seasons",
                       "Age":"first_age"}).reset_index()

In [369]:
pitches_injury = pitches_injury.merge(ps,on='Name')

In [370]:
pitches_injury = pitches_injury[pitches_injury['Num_Seasons'] > 1]
pitches_injury = pitches_injury[pitches_injury['first_age'] < 28]

In [371]:
pitches_injury = pitches_injury[pitches_injury['Season'] >= 2002]

In [372]:
pitches_injury["tj"] = [tj_injury(x,y) for x,y in pitches_injury[['Name','Season']].itertuples(index=False)]

In [373]:
pitches_injury["tj_year"] = [tj_injury_year(x,y) for x,y in pitches_injury[['Name','Season']].itertuples(index=False)]

## Pitching Injuries 2

In [374]:
pitches_injury['Injury'] = pitches_injury['Injury'].str.upper()

In [375]:
pitches_injury['injured'] = [1 if "ELBOW" in x
                             or "SHOULDER" in x
                             or "BACK" in x 
                             or "OBLIQUE" in x
                             or "ARM" in x 
                             or "KNEE" in x
                             or "FINGER" in x
                             or "GROIN" in x
                             or "HIP" in x
                             or "NECK" in x
                             else 0 for x in pitches_injury['Injury'].fillna("")]

In [376]:
# pitches_injury['injured_tag']  = [1 if (season <= injured_year) 
#                              and (injured == 1) and (season > (injured_year - 1))
# else 0 
# for name, season, injured_year, injured 
# in pitches_injury[['Name','Season','surgery_year','injured']].itertuples(index=False)]

In [377]:
pitches_injury = pitches_injury.drop_duplicates(["Name","Season","injured"])

In [378]:
pitches_injury = pitches_injury.sort_values("injured",ascending=False).drop_duplicates(subset=['Name','Season'])

In [379]:
pitch_types = ['FB%', 'SL%', 'CT%', 'CB%', 'CH%', 'SF%']
pitch_vels = ['FBv', 'SLv', 'CTv', 'CBv','CHv', 'SFv']

In [380]:
cols = ["injured",
 "FA-Z (pfx)",
 "FT-Z (pfx)",
 "FC-Z (pfx)",
 "FS-Z (pfx)",
 "FO-Z (pfx)",
 "SI-Z (pfx)",
 "SL-Z (pfx)",
 "CU-Z (pfx)",
 "KC-Z (pfx)",
 "EP-Z (pfx)",
 "CH-Z (pfx)",
 "SC-Z (pfx)",
 "KN-Z (pfx)"]

## Feature Engineering

In [381]:
def number_seasons_prior(Player, Season):
    return Season - pitches_injury[pitches_injury['Name'] == Player]['Season'].min()

In [382]:
pitches_injury['num_seasons_a_priori'] = [number_seasons_prior(x,y) 
                                 for x,y 
                                 in pitches_injury[['Name', 'Season']].itertuples(index=False)]

In [383]:
pitches_injury['seasons_until_injury'] = [x + 1 if y == 1 else 0 for x,y 
                                          in pitches_injury[['num_seasons_a_priori', 'injured']]
                                          .itertuples(index=False)]

In [384]:
for pitch_type in pitch_types:
    pitches_injury[pitch_type[:-1] + "_count"] = pitches_injury[pitch_type] * pitches_injury['Pitches']

In [385]:
def pitch_diff(Player, Season):
    temp_player = pitches_injury[(pitches_injury['Name'] == Player)
                                &(pitches_injury['Season'] <= Season)
                                & (pitches_injury['Season'] > Season - 3)]
    
    curr_player = pitches_injury[(pitches_injury['Name'] == Player)
                                &(pitches_injury['Season'] == Season)]
    
    temp_df = pd.DataFrame(temp_player[delta_types].mean()).T
    temp_df['Player'] = Player
    temp_df['Season'] = Season
    return temp_df

In [386]:
pitches_injury['pitches_per_game'] = pitches_injury['Pitches'] / pitches_injury['G']
pitches_injury["IP_per_G"] = pitches_injury['IP'] / pitches_injury['G']
pitches_injury["P_per_IP"] = pitches_injury['Pitches'] / pitches_injury['IP']


In [387]:
def shift_injury(player, season):
    if pitches_injury[(pitches_injury['Name'] == player) & 
                     (pitches_injury['Season'] == season + 1)]['injured'].sum() > 0 :
        return 1
    else:
        return 0

In [388]:
pitches_injury[(pitches_injury['Name'] == "JOHN LACKEY") & 
                     (pitches_injury['Season'] < 2017)]

,Season,Name,Team_x,Age,W,L,ERA,WAR,G,GS,CG,ShO,SV,BS,IP,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,GB,FB,LD,IFFB,Balls,Strikes,Pitches,RS,IFH,BU,BUH,K/9,BB/9,K/BB,H/9,HR/9,AVG,WHIP,BABIP,LOB%,FIP,GB/FB,LD%,GB%,IFFB%,HR/FB,IFH%,BUH%,Starting,Start-IP,Relieving,Relief-IP,RAR,Dollars,tERA,xFIP,WPA,-WPA,+WPA,RE24,REW,pLI,inLI,gmLI,exLI,Pulls,WPA/LI,Clutch,FB%,FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,HLD,SD,MD,ERA-,FIP-,xFIP-,K%,BB%,SIERA,RS/9,E-F,FA% (pfx),FT% (pfx),FC% (pfx),FS% (pfx),FO% (pfx),SI% (pfx),SL% (pfx),CU% (pfx),KC% (pfx),EP% (pfx),CH% (pfx),SC% (pfx),KN% (pfx),UN% (pfx),vFA (pfx),vFT (pfx),vFC (pfx),vFS (pfx),vFO (pfx),vSI (pfx),vSL (pfx),vCU (pfx),vKC (pfx),vEP (pfx),vCH (pfx),vSC (pfx),vKN (pfx),FA-X (pfx),FT-X (pfx),FC-X (pfx),FS-X (pfx),FO-X (pfx),SI-X (pfx),SL-X (pfx),CU-X (pfx),KC-X (pfx),EP-X (pfx),CH-X (pfx),SC-X (pfx),KN-X (pfx),FA-Z (pfx),FT-Z (pfx),FC-Z (pfx),FS-Z (pfx),FO-Z (pfx),SI-Z (pfx),SL-Z (pfx),CU-Z (pfx),KC-Z (pfx),EP-Z (pfx),CH-Z (pfx),SC-Z (pfx),KN-Z (pfx),wFA (pfx),wFT (pfx),wFC (pfx),wFS (pfx),wFO (pfx),wSI (pfx),wSL (pfx),wCU (pfx),wKC (pfx),wEP (pfx),wCH (pfx),wSC (pfx),wKN (pfx),wFA/C (pfx),wFT/C (pfx),wFC/C (pfx),wFS/C (pfx),wFO/C (pfx),wSI/C (pfx),wSL/C (pfx),wCU/C (pfx),wKC/C (pfx),wEP/C (pfx),wCH/C (pfx),wSC/C (pfx),wKN/C (pfx),O-Swing% (pfx),Z-Swing% (pfx),Swing% (pfx),O-Contact% (pfx),Z-Contact% (pfx),Contact% (pfx),Zone% (pfx),Pace,RA9-WAR,BIP-Wins,LOB-Wins,FDP-Wins,Age Rng,K-BB%,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,kwERA,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),Player,Team_y,Injury,Status,Dates,Days,Cash Earned On DL,Num_Seasons,first_age,tj,tj_year,injured,num_seasons_a_priori,seasons_until_injury,FB_count,SL_count,CT_count,CB_count,CH_count,SF_count,pitches_per_game,IP_per_G,P_per_IP
3911,2016.0,JOHN LACKEY,Cubs,37.0,11.0,8.0,3.35,2.9,29.0,29.0,0.0,0.0,0.0,0.0,188.1,748.0,146.0,74.0,70.0,23.0,53.0,1.0,9.0,4.0,0.0,180.0,201.0,178.0,111.0,15.0,981.0,1872.0,2853.0,108.0,15.0,16.0,3.0,8.60,2.53,3.40,6.98,1.10,0.213,1.06,0.255,0.762,3.81,1.13,0.227,0.410,0.084,0.129,0.075,0.188,27.6,188.1,NaN,NaN,27.6,$23.0,4.36,3.80,1.10,-12.17,13.27,22.70,2.34,0.98,0.94,0.87,1.01,29.0,1.69,-0.56,0.582,91.7,0.243,83.2,NaN,NaN,0.122,78.1,0.053,84.3,NaN,NaN,NaN,NaN,0.002,NaN,0.8,25.5,NaN,-2.0,-1.1,NaN,NaN,0.05,3.69,NaN,-0.58,-0.74,NaN,NaN,0.330,0.675,0.485,0.583,0.871,0.763,0.451,0.681,0.115,0.0,0.0,0.0,80.0,92.0,92.0,0.241,0.071,3.83,5.16,-0.47,0.373,0.209,0.232,NaN,NaN,NaN,NaN,0.133,NaN,NaN,0.054,NaN,NaN,NaN,91.8,91.6,83.4,NaN,NaN,NaN,NaN,78.3,NaN,NaN,84.2,NaN,NaN,-3.9,-8.8,2.9,NaN,NaN,NaN,NaN,4.3,NaN,NaN,-8.7,NaN,NaN,8.4,4.6,-0.3,NaN,NaN,NaN,NaN,-4.0,NaN,NaN,4.7,NaN,NaN,1.9,0.0,26.0,NaN,NaN,NaN,NaN,-2.1,NaN,NaN,-0.7,NaN,NaN,0.18,0.00,3.96,NaN,NaN,NaN,NaN,-0.57,NaN,NaN,-0.45,NaN,NaN,0.334,0.651,0.486,0.577,0.866,0.763,0.480,21.3,4.2,1.8,-0.5,1.3,37 - 37,0.170,0.376,0.360,0.265,0.162,0.494,0.344,3.71,0.342,0.054,NaN,0.115,0.424,NaN,NaN,NaN,NaN,0.158,0.250,NaN,84.9,NaN,78.6,92.5,NaN,NaN,NaN,NaN,92.3,83.7,NaN,-8.3,NaN,4.8,-3.9,NaN,NaN,NaN,NaN,-9.2,3.3,NaN,4.6,NaN,-4.6,8.2,NaN,NaN,NaN,NaN,3.8,-0.4,NaN,-0.7,NaN,0.6,4.3,NaN,NaN,NaN,NaN,-3.1,23.2,NaN,-0.44,N

In [389]:
def prior_injury(player, season):
    if pitches_injury[(pitches_injury['Name'] == player) & 
                     (pitches_injury['Season'] < season)]['injured'].sum() > 0 :
        return 1
    else:
        return 0

In [390]:
def prior_injury_last_2_seasons(player, season):
    if pitches_injury[(pitches_injury['Name'] == player) & 
                     (pitches_injury['Season'] < season) & 
                     (pitches_injury['Season'] >= season - 1)]['injured'].sum() > 0 :
        return 1
    else:
        return 0

In [391]:
pitches_injury['injury_lagged'] = [shift_injury(x,y) for x,y in pitches_injury[['Name','Season']].itertuples(index=False)]

In [392]:
pitches_injury['prior_injury'] = [prior_injury(x,y) for x,y in pitches_injury[['Name','Season']].itertuples(index=False)]

In [393]:
pitches_injury['prior_injury_last_2'] = [prior_injury_last_2_seasons(x,y) for x,y in pitches_injury[['Name','Season']].itertuples(index=False)]

In [394]:
pitches_injury.isnull().sum()

Season                      0
Name                        0
Team_x                      0
Age                         0
W                           0
L                           0
ERA                         0
WAR                         0
G                           0
GS                          0
CG                          0
ShO                         0
SV                          0
BS                          0
IP                          0
TBF                         0
H                           0
R                           0
ER                          0
HR                          0
BB                          0
IBB                         0
HBP                         0
WP                          0
BK                          0
SO                          0
GB                          0
FB                          0
LD                          0
IFFB                        0
Balls                       0
Strikes                     0
Pitches                     0
RS        

In [395]:
pitches_injury["FA% (pi)"]  =  [x if x == x 
                                else y if y == y
                                else z for x,y,z 
                                in pitches_injury[["FA% (pi)","FB%","FA% (pfx)"]].itertuples(index=False)]

In [396]:
pitches_injury["vFA (pi)"]  =  [x if x == x 
                                else y if y == y
                                else z for x,y,z 
                                in pitches_injury[["vFA (pi)","FBv","vFA (pfx)"]].itertuples(index=False)]

In [397]:
pitches_injury["FC% (pi)"]  =  [x if x == x 
                                else y if y == y
                                else z for x,y,z 
                                in pitches_injury[["FC% (pi)","CT%","FC% (pfx)"]].itertuples(index=False)]

In [398]:
pitches_injury["vFC (pi)"]  =  [x if x == x 
                                else y if y == y
                                else z for x,y,z 
                                in pitches_injury[["vFC (pi)","CTv","vFC (pfx)"]].itertuples(index=False)]

In [399]:
pitches_injury["SI% (pi)"]  =  [x if x == x 
                                else y for x,y 
                                in pitches_injury[["SI% (pi)","SI% (pfx)"]].itertuples(index=False)]

In [400]:
pitches_injury["vSI (pi)"]  =  [x if x == x 
                                else y for x,y 
                                in pitches_injury[["vSI (pi)","vSI (pfx)"]].itertuples(index=False)]

In [401]:
pitches_injury["FA-Z (pi)"]  =  [x if x == x 
                                else y for x,y
                                in pitches_injury[["FA-Z (pi)","FA-Z (pfx)"]].itertuples(index=False)]

pitches_injury["FA-X (pi)"]  =  [x if x == x 
                                else y for x,y
                                in pitches_injury[["FA-X (pi)","FA-X (pfx)"]].itertuples(index=False)]

pitches_injury["wFA (pi)"]  =  [x if x == x 
                                else y for x,y
                                in pitches_injury[["wFA (pi)","wFA (pfx)"]].itertuples(index=False)]

In [402]:
pitches_injury["SI-Z (pi)"]  =  [x if x == x 
                                else y for x,y
                                in pitches_injury[["SI-Z (pi)","SI-Z (pfx)"]].itertuples(index=False)]

pitches_injury["SI-X (pi)"]  =  [x if x == x 
                                else y for x,y
                                in pitches_injury[["SI-X (pi)","SI-X (pfx)"]].itertuples(index=False)]

pitches_injury["wSI (pi)"]  =  [x if x == x 
                                else y for x,y
                                in pitches_injury[["wSI (pi)","wSI (pfx)"]].itertuples(index=False)]

In [403]:
pitches_injury["FC-Z (pi)"]  =  [x if x == x 
                                else y for x,y
                                in pitches_injury[["FC-Z (pi)","FC-Z (pfx)"]].itertuples(index=False)]

pitches_injury["FC-X (pi)"]  =  [x if x == x 
                                else y for x,y
                                in pitches_injury[["FC-X (pi)","FC-X (pfx)"]].itertuples(index=False)]

pitches_injury["wFC (pi)"]  =  [x if x == x 
                                else y for x,y
                                in pitches_injury[["wFC (pi)","wFC (pfx)"]].itertuples(index=False)]

In [404]:
pitch_types = ['CH% (pi)', 'CS% (pi)', 'CU% (pi)', 'FA% (pi)',
               'FC% (pi)', 'FS% (pi)', 'KN% (pi)', 'SB% (pi)',
               'SI% (pi)', 'SL% (pi)', 'XX% (pi)',
              'CH-X (pi)', 'CS-X (pi)', 'CU-X (pi)', 'FA-X (pi)',
               'FC-X (pi)', 'FS-X (pi)', 'KN-X (pi)', 'SB-X (pi)',
               'SI-X (pi)', 'SL-X (pi)', 'XX-X (pi)',
               'CH-Z (pi)', 'CS-Z (pi)', 'CU-Z (pi)', 'FA-Z (pi)',
               'FC-Z (pi)', 'FS-Z (pi)', 'KN-Z (pi)', 'SB-Z (pi)',
               'SI-Z (pi)', 'SL-Z (pi)', 'XX-Z (pi)',
               'wCH (pi)', 'wCS (pi)', 'wCU (pi)',
               'wFA (pi)', 'wFC (pi)', 'wFS (pi)',
               'wKN (pi)', 'wSB (pi)', 'wSI (pi)',
               'wSL (pi)', 'wXX (pi)'
               
              ]

In [405]:
pitches_injury['sinkers_cutters_percent'] = pitches_injury['FC% (pi)'].fillna(0) + pitches_injury['SI% (pi)'].fillna(0)

In [406]:
pitches_injury['other_pitch_types_percent'] = (pitches_injury['CH% (pi)'].fillna(0) +
                                       pitches_injury['CS% (pi)'].fillna(0) +
                                       pitches_injury['CU% (pi)'].fillna(0) +
                                       pitches_injury['FS% (pi)'].fillna(0) +
                                       pitches_injury['KN% (pi)'].fillna(0) +
                                       pitches_injury['SB% (pi)'].fillna(0) +
                                       pitches_injury['SL% (pi)'].fillna(0) +
                                       pitches_injury['XX% (pi)'].fillna(0)
                                      )

In [407]:
pitches_injury['other_pitch_types_w'] = (pitches_injury['wCH (pi)'].fillna(0) +
                                       pitches_injury['wCS (pi)'].fillna(0) +
                                       pitches_injury['wCU (pi)'].fillna(0) +
                                       pitches_injury['wFS (pi)'].fillna(0) +
                                       pitches_injury['wKN (pi)'].fillna(0) +
                                       pitches_injury['wSB (pi)'].fillna(0) +
                                       pitches_injury['wSL (pi)'].fillna(0) +
                                       pitches_injury['wXX (pi)'].fillna(0)
                                      )

In [408]:
pitches_injury['other_pitch_types_X'] = (pitches_injury['CH-X (pi)'].fillna(0) +
                                       pitches_injury['CS-X (pi)'].fillna(0) +
                                       pitches_injury['CU-X (pi)'].fillna(0) +
                                       pitches_injury['FS-X (pi)'].fillna(0) +
                                       pitches_injury['KN-X (pi)'].fillna(0) +
                                       pitches_injury['SB-X (pi)'].fillna(0) +
                                       pitches_injury['SL-X (pi)'].fillna(0) +
                                       pitches_injury['XX-X (pi)'].fillna(0)
                                      )

In [409]:
pitches_injury['other_pitch_types_Z'] = (pitches_injury['CH-Z (pi)'].fillna(0) +
                                       pitches_injury['CS-Z (pi)'].fillna(0) +
                                       pitches_injury['CU-Z (pi)'].fillna(0) +
                                       pitches_injury['FS-Z (pi)'].fillna(0) +
                                       pitches_injury['KN-Z (pi)'].fillna(0) +
                                       pitches_injury['SB-Z (pi)'].fillna(0) +
                                       pitches_injury['SL-Z (pi)'].fillna(0) +
                                       pitches_injury['XX-Z (pi)'].fillna(0)
                                      )

In [410]:
others = ["other_pitch_types_percent","other_pitch_types_w","other_pitch_types_X","other_pitch_types_Z"]

In [411]:
def calc_accel(df, field):
    temp = (df
                   .groupby(['Name','Season'])[field]
                   .sum().groupby(level=0, group_keys=False)
                   .shift(+2).reset_index())

    temp.rename(columns={field:"{}_lag_2".format(field)}, inplace = True)

    if "{}_lag_2".format(field) in df.columns:
        df.drop(columns=["{}_lag_2".format(field)],inplace=True)
    
    df = pd.merge(df, temp, on=['Name','Season'])

    df['{}_diff_2'.format(field)] = df[field] - df["{}_lag_2".format(field)]
    df['{}_accel_2'.format(field)] = df['{}_diff_2'.format(field)] / 3
    return df

In [412]:
pitches_injury['FA% (pi)']

9834     0.279
10547    0.417
11768    0.450
11769    0.460
11771    0.413
         ...  
5690     0.470
5691     0.508
5692     0.780
5693     0.374
13147    0.482
Name: FA% (pi), Length: 10094, dtype: float64

In [413]:
pitches_injury['years_since_tj'] = [y - x if x > 0 
                                    else 0 
                                    for x,y 
                                    in pitches_injury[['tj_year',"Season"]].itertuples(index=False)]

In [414]:
cols = ['Pitches','G', "IP", "sinkers_cutters_percent",'vFA (pi)','vFC (pi)',"vSI (pi)","FA% (pi)"]

In [415]:
pitches_injury['SO_G'] = pitches_injury['SO'] / pitches_injury['G']

In [416]:
## Generate acceleration features

In [417]:
for col in cols:
    pitches_injury = calc_accel(pitches_injury, col)

In [418]:
for col in pitch_types:
    pitches_injury = calc_accel(pitches_injury, col)

In [419]:
for col in others:
    pitches_injury = calc_accel(pitches_injury, col)

In [420]:
pitches_injury = calc_accel(pitches_injury, 'SO_G')

In [421]:
pitches_injury = calc_accel(pitches_injury, 'P_per_IP')

In [422]:
pitches_injury = calc_accel(pitches_injury, 'vSL (pi)')

TBF, BB, WP

In [423]:
other_stats = ["walk_rate","wild_pitch_rate"]

In [424]:
pitches_injury['walk_rate'] = ((pitches_injury['BB'] - pitches_injury['IBB']) / pitches_injury['TBF'])

In [425]:
pitches_injury['wild_pitch_rate'] = (pitches_injury['WP'] / pitches_injury['Pitches'])

In [426]:
pitches_injury[pitches_injury['IBB'] == pitches_injury['BB']][['Name','Team_x','Season','BB','IBB']]

,Name,Team_x,Season,BB,IBB
296,JERAD EICKHOFF,Phillies,2018.0,0.0,0.0
319,EVAN SCRIBNER,Mariners,2017.0,0.0,0.0
358,CRISTHIAN MARTINEZ,Braves,2013.0,0.0,0.0
492,BRIAN FLYNN,Royals,2017.0,0.0,0.0
515,TAIJUAN WALKER,Diamondbacks,2019.0,0.0,0.0
547,RICARDO RODRIGUEZ,Rangers,2018.0,1.0,1.0
673,SHAE SIMMONS,Braves,2016.0,0.0,0.0
790,JERRY BLEVINS,Mets,2015.0,0.0,0.0
844,JORDAN MONTGOMERY,Yankees,2019.0,0.0,0.0
874,ANTHONY BANDA,Rays,2019.0,0.0,0.0


In [427]:
pitches_injury['win_rec'] = (pitches_injury['W'] / (pitches_injury['L'] + pitches_injury['W'])).fillna(0)

In [428]:
for col in other_stats:
    pitches_injury = calc_accel(pitches_injury, col)

In [471]:
more_stats = ["wFC/C (pi)","wSI/C (pi)","wFA/C (pi)"]

In [472]:
for col in more_stats:
    pitches_injury = calc_accel(pitches_injury, col)

In [429]:
(pitches_injury[pitches_injury['Season'] > 2013]
 .groupby([     
     'tj',
     #pd.cut(pitches_injury[pitches_injury['Season'] > 2013]["Age"], 5, duplicates="drop"),
           pd.cut(pitches_injury[pitches_injury['Season'] > 2013]['wKN (pi)_accel_2'], bins=2,duplicates="drop"),
     
          ])
 .agg({"injury_lagged":['mean','sum','count']}))

injury_lagged           
                             mean  sum count
tj wKN (pi)_accel_2                         
0  (-9.112, -3.083]          0.00  0.0   2.0
   (-3.083, 2.933]           0.25  1.0   4.0
1  (-9.112, -3.083]           NaN  NaN   NaN
   (-3.083, 2.933]           0.00  0.0   2.0

In [430]:
pitch_types = ['CH% (pi)', 'CS% (pi)', 'CU% (pi)', 'FA% (pi)',
               'FC% (pi)', 'FS% (pi)', 'KN% (pi)', 'SB% (pi)',
               'SI% (pi)', 'SL% (pi)', 'XX% (pi)',
              'CH-X (pi)', 'CS-X (pi)', 'CU-X (pi)', 'FA-X (pi)',
               'FC-X (pi)', 'FS-X (pi)', 'KN-X (pi)', 'SB-X (pi)',
               'SI-X (pi)', 'SL-X (pi)', 'XX-X (pi)',
               'CH-Z (pi)', 'CS-Z (pi)', 'CU-Z (pi)', 'FA-Z (pi)',
               'FC-Z (pi)', 'FS-Z (pi)', 'KN-Z (pi)', 'SB-Z (pi)',
               'SI-Z (pi)', 'SL-Z (pi)', 'XX-Z (pi)',
               'wCH (pi)', 'wCS (pi)', 'wCU (pi)',
               'wFA (pi)', 'wFC (pi)', 'wFS (pi)',
               'wKN (pi)', 'wSB (pi)', 'wSI (pi)',
               'wSL (pi)', 'wXX (pi)',
               'CH% (pi)_accel_2', 'CS% (pi)_accel_2', 'CU% (pi)_accel_2', 'FA% (pi)_accel_2',
               'FC% (pi)_accel_2', 'FS% (pi)_accel_2', 'KN% (pi)_accel_2', 'SB% (pi)_accel_2',
               'SI% (pi)_accel_2', 'SL% (pi)_accel_2', 'XX% (pi)_accel_2',
              'CH-X (pi)_accel_2', 'CS-X (pi)_accel_2', 'CU-X (pi)_accel_2', 'FA-X (pi)_accel_2',
               'FC-X (pi)_accel_2', 'FS-X (pi)_accel_2', 'KN-X (pi)_accel_2', 'SB-X (pi)_accel_2',
               'SI-X (pi)_accel_2', 'SL-X (pi)_accel_2', 'XX-X (pi)_accel_2',
               'CH-Z (pi)_accel_2', 'CS-Z (pi)_accel_2', 'CU-Z (pi)_accel_2', 'FA-Z (pi)_accel_2',
               'FC-Z (pi)_accel_2', 'FS-Z (pi)_accel_2', 'KN-Z (pi)_accel_2', 'SB-Z (pi)_accel_2',
               'SI-Z (pi)_accel_2', 'SL-Z (pi)_accel_2', 'XX-Z (pi)_accel_2',
               'wCH (pi)_accel_2', 'wCS (pi)_accel_2', 'wCU (pi)_accel_2',
               'wFA (pi)_accel_2', 'wFC (pi)_accel_2', 'wFS (pi)_accel_2',
               'wKN (pi)_accel_2', 'wSB (pi)_accel_2', 'wSI (pi)_accel_2',
               'wSL (pi)_accel_2', 'wXX (pi)_accel_2'
               
              ]

In [431]:
pitch_types_accel = ['CH% (pi)_accel_2', 'CS% (pi)_accel_2', 'CU% (pi)_accel_2', 'FA% (pi)_accel_2',
               'FC% (pi)_accel_2', 'FS% (pi)_accel_2', 'KN% (pi)_accel_2', 'SB% (pi)_accel_2',
               'SI% (pi)_accel_2', 'SL% (pi)_accel_2', 'XX% (pi)_accel_2',
              'CH-X (pi)_accel_2', 'CS-X (pi)_accel_2', 'CU-X (pi)_accel_2', 'FA-X (pi)_accel_2',
               'FC-X (pi)_accel_2', 'FS-X (pi)_accel_2', 'KN-X (pi)_accel_2', 'SB-X (pi)_accel_2',
               'SI-X (pi)_accel_2', 'SL-X (pi)_accel_2', 'XX-X (pi)_accel_2',
               'CH-Z (pi)_accel_2', 'CS-Z (pi)_accel_2', 'CU-Z (pi)_accel_2', 'FA-Z (pi)_accel_2',
               'FC-Z (pi)_accel_2', 'FS-Z (pi)_accel_2', 'KN-Z (pi)_accel_2', 'SB-Z (pi)_accel_2',
               'SI-Z (pi)_accel_2', 'SL-Z (pi)_accel_2', 'XX-Z (pi)_accel_2',
               'wCH (pi)_accel_2', 'wCS (pi)_accel_2', 'wCU (pi)_accel_2',
               'wFA (pi)_accel_2', 'wFC (pi)_accel_2', 'wFS (pi)_accel_2',
               'wKN (pi)_accel_2', 'wSB (pi)_accel_2', 'wSI (pi)_accel_2',
               'wSL (pi)_accel_2', 'wXX (pi)_accel_2'
              ]

In [432]:
pitch_types = ['CH-X (pi)', 'CS-X (pi)', 'CU-X (pi)', 'FA-X (pi)',
               'FC-X (pi)', 'FS-X (pi)', 'KN-X (pi)', 'SB-X (pi)',
               'SI-X (pi)', 'SL-X (pi)', 'XX-X (pi)']

In [433]:
pitches_injury[['vCH (pi)','vCS (pi)','vCU (pi)','vFA (pi)',
              'vFC (pi)','vFS (pi)','vKN (pi)','vSB (pi)',
              'vSI (pi)','vSL (pi)','vXX (pi)']]

,vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi)
0,83.6,NaN,78.8,92.9,82.4,NaN,NaN,NaN,92.0,81.5,NaN
1,84.0,NaN,NaN,91.8,87.5,NaN,NaN,NaN,91.2,85.1,NaN
2,87.0,NaN,79.8,94.7,85.4,NaN,NaN,NaN,NaN,84.3,NaN
3,84.9,NaN,79.3,91.5,89.3,NaN,NaN,NaN,90.9,85.2,NaN
4,88.5,NaN,82.3,94.7,NaN,NaN,NaN,NaN,93.8,86.1,NaN
...,...,...,...,...,...,...,...,...,...,...,...
10089,84.1,NaN,81.3,92.3,85.9,NaN,NaN,NaN,91.8,82.7,NaN
10090,83.2,NaN,80.1,91.9,85.2,NaN,NaN,NaN,91.2,82.3,NaN
10091,81.7,NaN,NaN,88.4,NaN,NaN,NaN,NaN,89.6,81.9,NaN
10092,84.7,NaN,NaN,90.4,NaN,NaN,NaN,NaN,90.1,82.3,NaN


## World Series?

In [434]:
ws_teams = {"Red Sox":2013,
"Cardinals":2013,
"Giants":2014,
"Royals":2014,
"Royals":2015,
"Mets":2015,
"Cubs":2016,
"Indians":2016,
"Astros":2017,
"Dodgers":2017,
"Red Sox":2018,
"Dodgers":2018,
"Nationals":2019,
"Astros":2019,
}

In [435]:
pitches_injury['ws_pitcher'] = [1 if y in ws_teams 
 and ws_teams[y] == x 
 else 0 
 for x,y in pitches_injury[['Season','Team_x']].itertuples(index=False)]

In [436]:
## Acceleration over a three year period would be the final prop - initial prop / 3. The Acceleration would then be
## the rate at which the proportion increased over the three year period. 

In [437]:
pitches_injury['pitches_per_bat'] = (pitches_injury['Pitches'] / pitches_injury['TBF'])

In [438]:
pitches_injury[pitches_injury['Name'] == "CLAYTON KERSHAW"][["Name","Season",
                                                                      'FA% (pi)',"FA% (pi)_accel_2"]].sort_values("Season")

,Name,Season,FA% (pi),FA% (pi)_accel_2
2272,CLAYTON KERSHAW,2008.0,0.713,NaN
2271,CLAYTON KERSHAW,2009.0,0.701,NaN
2270,CLAYTON KERSHAW,2010.0,0.716,0.001000
2269,CLAYTON KERSHAW,2011.0,0.653,-0.016000
2268,CLAYTON KERSHAW,2012.0,0.621,-0.031667
2267,CLAYTON KERSHAW,2013.0,0.606,-0.015667
951,CLAYTON KERSHAW,2014.0,0.553,-0.022667
2266,CLAYTON KERSHAW,2015.0,0.538,-0.022667
967,CLAYTON KERSHAW,2016.0,0.508,-0.015000
968,CLAYTON KERSHAW,2017.0,0.466,-0.024000


In [476]:
training_cols = ["Name","Season","injured","injury_lagged", "Age",'Pitches','IP','G',"P_per_IP_accel_2",
                'prior_injury','sinkers_cutters_percent','prior_injury_last_2','P_per_IP',"pitches_per_bat",
                 'Pitches_accel_2','G_accel_2','IP_accel_2',"tj","sinkers_cutters_percent_accel_2","IP_per_G",
                 "pitches_per_game",
                 "other_pitch_types_percent_accel_2","other_pitch_types_w_accel_2","years_since_tj",
                 "other_pitch_types_X_accel_2","other_pitch_types_Z_accel_2","SO_G_accel_2","ws_pitcher",
                 "walk_rate","walk_rate_accel_2","wild_pitch_rate","wild_pitch_rate_accel_2",
                 'FA% (pi)','FC% (pi)','SI% (pi)',
              'FA-X (pi)','FC-X (pi)','SI-X (pi)',
                'FA-Z (pi)','FC-Z (pi)','SI-Z (pi)',
               'wFA (pi)', 'wFC (pi)', 'wSI (pi)',
                'FA% (pi)_accel_2',
               'FC% (pi)_accel_2',
               'SI% (pi)_accel_2',
               'FA-X (pi)_accel_2',
               'FC-X (pi)_accel_2',
               'SI-X (pi)_accel_2',
               'FA-Z (pi)_accel_2',
               'FC-Z (pi)_accel_2','wFC/C (pi)','wFC/C (pi)_accel_2',
                 'wSI/C (pi)','wSI/C (pi)_accel_2','wFA/C (pi)','wFA/C (pi)_accel_2',
               'SI-Z (pi)_accel_2',"win_rec",
               'wFA (pi)_accel_2', 'wSI (pi)_accel_2','wFC (pi)_accel_2',
                 'vFA (pi)','vFC (pi)','vSI (pi)','vFC (pi)_accel_2','vFA (pi)_accel_2','vSI (pi)_accel_2',
                 'SL% (pi)',
                 'SL% (pi)_accel_2',
                 'SL-X (pi)',
                 'SL-X (pi)_accel_2',
                 'SL-Z (pi)',
                 'SL-Z (pi)_accel_2',
                'vSL (pi)',
                 'vSL (pi)_accel_2',
                    'wSL (pi)',
                 'wSL (pi)_accel_2'
    
                ]

In [478]:
pitches_injury_2015['FA% (pi)']

5        0.145
11       0.007
14       0.517
17       0.121
18       0.026
20       0.544
22       0.548
27       0.927
28       0.843
29       0.542
32       0.934
35       0.371
39       0.406
41       0.060
42       0.341
44       0.202
45       0.384
47       0.475
51       0.495
53       0.018
54       0.511
55       0.430
60       0.279
64       0.703
69       0.064
70       0.484
72       0.605
73       0.380
77       0.591
80       0.370
81       0.085
82       0.381
83       0.452
90       0.381
97       0.231
103      0.495
110      0.380
111      0.274
112      0.327
114      0.343
115      0.334
116      0.584
119      0.136
121      0.418
122      0.365
123      0.579
124      0.450
126      0.470
127      0.442
128      0.313
130      0.431
133      0.598
138      0.544
140      0.388
141      0.455
143      0.547
144      0.347
146      0.771
147      0.406
149      0.452
151      0.381
158      0.659
165      0.409
166      0.260
167      0.499
169      0.627
170      0

In [479]:
pitches_injury_2015.loc[1765][['FA% (pfx)',"FB%","FA% (pi)"]]

KeyError: 1765

In [480]:
pitches_injury_2015[pitches_injury_2015['vFA (pi)'].isnull()][["IP",'vFA (pi)',"FB%","FA% (pi)","Season","Team_x",
                                                              "injury_lagged"]]

,IP,vFA (pi),FB%,FA% (pi),Season,Team_x,injury_lagged
1324,66.2,NaN,NaN,NaN,2016.0,Indians,0


In [481]:
train = ""
for each in training_cols:
    train = train + " + " + each

In [482]:
train.replace("%",".").replace("(",".").replace(")",".").replace(" ",".")

'.+.Name.+.Season.+.injured.+.injury_lagged.+.Age.+.Pitches.+.IP.+.G.+.P_per_IP_accel_2.+.prior_injury.+.sinkers_cutters_percent.+.prior_injury_last_2.+.P_per_IP.+.pitches_per_bat.+.Pitches_accel_2.+.G_accel_2.+.IP_accel_2.+.tj.+.sinkers_cutters_percent_accel_2.+.IP_per_G.+.pitches_per_game.+.other_pitch_types_percent_accel_2.+.other_pitch_types_w_accel_2.+.years_since_tj.+.other_pitch_types_X_accel_2.+.other_pitch_types_Z_accel_2.+.SO_G_accel_2.+.ws_pitcher.+.walk_rate.+.walk_rate_accel_2.+.wild_pitch_rate.+.wild_pitch_rate_accel_2.+.FA...pi..+.FC...pi..+.SI...pi..+.FA-X..pi..+.FC-X..pi..+.SI-X..pi..+.FA-Z..pi..+.FC-Z..pi..+.SI-Z..pi..+.wFA..pi..+.wFC..pi..+.wSI..pi..+.FA...pi._accel_2.+.FC...pi._accel_2.+.SI...pi._accel_2.+.FA-X..pi._accel_2.+.FC-X..pi._accel_2.+.SI-X..pi._accel_2.+.FA-Z..pi._accel_2.+.FC-Z..pi._accel_2.+.wFC/C..pi..+.wFC/C..pi._accel_2.+.wSI/C..pi..+.wSI/C..pi._accel_2.+.wFA/C..pi..+.wFA/C..pi._accel_2.+.SI-Z..pi._accel_2.+.win_rec.+.wFA..pi._accel_2.+.wSI..pi._acce

In [483]:
pitches_injury_2013 = pitches_injury[pitches_injury['Season'] > 2013]
pitches_injury_2015 = pitches_injury[(pitches_injury['Season'] > 2013) & (pitches_injury['Season'] < 2018)]
pitches_injury_2019 =  pitches_injury[pitches_injury['Season'] == 2018]
pitches_injury_2020 = pitches_injury[pitches_injury['Season'] == 2019]

In [484]:
pitches_injury_2015 = pitches_injury_2015[(pitches_injury_2015['IP'] > 10)]

In [485]:
## Removing Pitchers who don't have a sufficient history to create acceleration variables. 
pitches_injury_2015 = pitches_injury_2015[~pitches_injury_2015['SO_G_accel_2'].isnull()]

In [486]:
pitches_injury_2015[training_cols].fillna(0).set_index("Name",drop=True).to_csv("/Users/nchapman/Documents/Documents/MSAS/spring_2020/MAT8406/Pitcher Injury Project/pitching_data.csv",
                     sep='|')
pitches_injury_2019[training_cols].fillna(0).set_index("Name", drop=True).to_csv("/Users/nchapman/Documents/Documents/MSAS/spring_2020/MAT8406/Pitcher Injury Project/pitching_holdout_data.csv",
                                                                                sep='|')
pitches_injury_2020[training_cols].fillna(0).set_index("Name", drop=True).to_csv("/Users/nchapman/Documents/Documents/MSAS/spring_2020/MAT8406/Pitcher Injury Project/pitching_holdout_2020_data.csv",
                                                                                sep='|')

In [487]:
pitches_injury_2015.groupby("injury_lagged").agg({"Age":"mean",
                                           "tj":"mean",
                                           "prior_injury":"mean",
                                           "win_rec":"mean",
                                           "IP":"mean",
                                            "ws_pitcher":"mean",
                                                 "pitches_per_bat":"mean",
                                                 "pitches_per_game":"mean"})

,Age,tj,prior_injury,win_rec,IP,ws_pitcher,pitches_per_bat,pitches_per_game
injury_lagged,,,,,,,,
0,29.405757,0.141133,0.375116,0.463073,80.101021,0.038997,3.875158,45.271916
1,29.047414,0.200431,0.476293,0.515199,93.285345,0.032328,3.868595,52.143339


In [451]:
pitches_injury_2019.groupby("injured").agg({"Age":"mean",
                                           "tj":"mean",
                                           "prior_injury":"mean",
                                           "win_rec":"mean",
                                           "IP":"mean",
                                            "ws_pitcher":"mean",
                                                 "walk_rate":"mean",
                                           "pitches_per_bat":"mean",
                                           "pitches_per_game":"mean"})

,Age,tj,prior_injury,win_rec,IP,ws_pitcher,walk_rate,pitches_per_bat,pitches_per_game
injured,,,,,,,,,
0,27.294492,0.069915,0.322034,0.393254,58.812076,0.040254,0.091178,3.904123,39.880807
1,28.593607,0.178082,0.611872,0.405318,63.068037,0.091324,0.082516,3.893039,47.208188


In [452]:
Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -2.014e+00  1.518e-01 -13.268   <2e-16 ***
IP           2.191e-02  8.935e-03   2.452   0.0142 *  
Pitches     -6.718e-04  5.252e-04  -1.279   0.2008    
IP:Pitches  -2.318e-06  1.063e-06  -2.181   0.0292 *  

SyntaxError: invalid syntax (<ipython-input-452-8825e638060e>, line 1)

In [453]:
pitches_injury.sample(2)[['Name',"Season",'Pitches','IP']]

,Name,Season,Pitches,IP
4647,JON EDWARDS,2015.0,315.0,16.2
6469,ROB BELL,2004.0,1885.0,123.0


In [454]:
pitches_injury.sort_values("Season",inplace=True)


In [455]:
injury_lagged ~ Pitches + prior_injury + P_per_IP + 
    IP_accel_2 + wSI..pi._accel_2 + wFA..pi._accel_2 + walk_rate + 
    win_rec + vFA..pi. + FA...pi. + tj + FA.Z..pi. + wild_pitch_rate_accel_2 + 
    P_per_IP:IP_accel_2 + FA...pi.:FA.Z..pi. + P_per_IP:wFA..pi._accel_2 + 
    wSI..pi._accel_2:wFA..pi._accel_2 + tj:wild_pitch_rate_accel_2

SyntaxError: invalid syntax (<ipython-input-455-702f85bee111>, line 1)

In [456]:
model_cols = ["Name","Season","IP","injury_lagged","injured","Pitches","prior_injury","P_per_IP","IP_per_G",
              "IP_accel_2","wSI (pi)_accel_2","wFA (pi)_accel_2","P_per_IP_accel_2",
             "walk_rate","win_rec","vFA (pi)","tj","FA% (pi)", "FA-Z (pi)","wild_pitch_rate_accel_2"]

In [489]:
pitches_injury[["Name","Season","wSI (pi)_accel_2",'wFB',"wFA (pfx)",
                "wFC/C (pi)","wSI/C (pi)","wFA/C (pi)"]].isnull().sum()

Name                   0
Season                 0
wSI (pi)_accel_2    5975
wFB                   12
wFA (pfx)           2441
wFC/C (pi)          7920
wSI/C (pi)          4672
wFA/C (pi)          2711
dtype: int64

In [462]:
len(pitches_injury)

10094

In [468]:
pitches_injury[(pitches_injury['Name'] == 'CLAYTON KERSHAW')]

,Season,Name,Team_x,Age,W,L,ERA,WAR,G,GS,CG,ShO,SV,BS,IP,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,GB,FB,LD,IFFB,Balls,Strikes,Pitches,RS,IFH,BU,BUH,K/9,BB/9,K/BB,H/9,HR/9,AVG,WHIP,BABIP,LOB%,FIP,GB/FB,LD%,GB%,IFFB%,HR/FB,IFH%,BUH%,Starting,Start-IP,Relieving,Relief-IP,RAR,Dollars,tERA,xFIP,WPA,-WPA,+WPA,RE24,REW,pLI,inLI,gmLI,exLI,Pulls,WPA/LI,Clutch,FB%,FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,HLD,SD,MD,ERA-,FIP-,xFIP-,K%,BB%,SIERA,RS/9,E-F,FA% (pfx),FT% (pfx),FC% (pfx),FS% (pfx),FO% (pfx),SI% (pfx),SL% (pfx),CU% (pfx),KC% (pfx),EP% (pfx),CH% (pfx),SC% (pfx),KN% (pfx),UN% (pfx),vFA (pfx),vFT (pfx),vFC (pfx),vFS (pfx),vFO (pfx),vSI (pfx),vSL (pfx),vCU (pfx),vKC (pfx),vEP (pfx),vCH (pfx),vSC (pfx),vKN (pfx),FA-X (pfx),FT-X (pfx),FC-X (pfx),FS-X (pfx),FO-X (pfx),SI-X (pfx),SL-X (pfx),CU-X (pfx),KC-X (pfx),EP-X (pfx),CH-X (pfx),SC-X (pfx),KN-X (pfx),FA-Z (pfx),FT-Z (pfx),FC-Z (pfx),FS-Z (pfx),FO-Z (pfx),SI-Z (pfx),SL-Z (pfx),CU-Z (pfx),KC-Z (pfx),EP-Z (pfx),CH-Z (pfx),SC-Z (pfx),KN-Z (pfx),wFA (pfx),wFT (pfx),wFC (pfx),wFS (pfx),wFO (pfx),wSI (pfx),wSL (pfx),wCU (pfx),wKC (pfx),wEP (pfx),wCH (pfx),wSC (pfx),wKN (pfx),wFA/C (pfx),wFT/C (pfx),wFC/C (pfx),wFS/C (pfx),wFO/C (pfx),wSI/C (pfx),wSL/C (pfx),wCU/C (pfx),wKC/C (pfx),wEP/C (pfx),wCH/C (pfx),wSC/C (pfx),wKN/C (pfx),O-Swing% (pfx),Z-Swing% (pfx),Swing% (pfx),O-Contact% (pfx),Z-Contact% (pfx),Contact% (pfx),Zone% (pfx),Pace,RA9-WAR,BIP-Wins,LOB-Wins,FDP-Wins,Age Rng,K-BB%,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,kwERA,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),Player,Team_y,Injury,Status,Dates,Days,Cash Earned On DL,Num_Seasons,first_age,tj,tj_year,injured,num_seasons_a_priori,seasons_until_injury,FB_count,SL_count,CT_count,CB_count,CH_count,SF_count,pitches_per_game,IP_per_G,P_per_IP,injury_lagged,prior_injury,prior_injury_last_2,sinkers_cutters_percent,other_pitch_types_percent,other_pitch_types_w,other_pitch_types_X,other_pitch_types_Z,years_since_tj,SO_G,Pitches_lag_2,Pitches_diff_2,Pitches_accel_2,G_lag_2,G_diff_2,G_accel_2,IP_lag_2,IP_diff_2,IP_accel_2,sinkers_cutters_percent_lag_2,sinkers_cutters_percent_diff_2,sinkers_cutters_percent_accel_2,vFA (pi)_lag_2,vFA (pi)_diff_2,vFA (pi)_accel_2,vFC (pi)_lag_2,vFC (pi)_diff_2,vFC (pi)_accel_2,vSI (pi)_lag_2,vSI (pi)_diff_2,vSI (pi)_accel_2,FA% (pi)_diff_2,FA% (pi)_accel_2,CH% (pi)_lag_2,CH% (pi)_diff_2,CH% (pi)_accel_2,CS% (pi)_lag_2,CS% (pi)_diff_2,CS% (pi)_accel_2,CU% (pi)_lag_2,CU% (pi)_diff_2,CU% (pi)_accel_2,FA% (pi)_lag_2,FC% (pi)_lag_2,FC% (pi)_diff_2,FC% (pi)_accel_2,FS% (pi)_lag_2,FS% (pi)_diff_2,FS% (pi)_accel_2,KN% (pi)_lag_2,KN% (pi)_diff_2,KN% (pi)_accel_2,SB% (pi)_lag_2,SB% (pi)_diff_2,SB% (pi)_accel_2,SI% (pi)_lag_2,SI% (pi)_diff_2,SI% (pi)_accel_2,SL% (pi)_lag_2,SL% (pi)_diff_2,SL% (pi)_accel_2,XX% (pi)_lag_2,XX% (pi)_diff_2,XX% (pi)_accel_2,CH-X (pi)_lag_2,CH-X (pi)_diff_2,CH-X (pi)_accel_2,CS-X (pi)_lag_2,CS-X (pi)_diff_2,CS-X (pi)_accel_2,CU-X (pi)_lag_2,CU-X (pi)_diff_2,CU-X (pi)_accel_2,FA-X (pi)_lag_2,FA-X (pi)_diff_2,FA-X (pi)_accel_2,FC-X (pi)_lag_2,FC-X (pi)_diff_2,FC-X (pi)_accel_2,FS-X (pi)_lag_2,FS-X (pi)_diff_2,FS-X (pi)_accel_2,KN-X (pi)_lag_2,KN-X (p

In [327]:
pitches_injury[(pitches_injury['Name'] == 'DAVID PRICE')][training_cols]

,Name,Season,injured,injury_lagged,Age,Pitches,IP,G,P_per_IP_accel_2,prior_injury,sinkers_cutters_percent,prior_injury_last_2,P_per_IP,pitches_per_bat,Pitches_accel_2,G_accel_2,IP_accel_2,tj,sinkers_cutters_percent_accel_2,IP_per_G,pitches_per_game,other_pitch_types_percent_accel_2,other_pitch_types_w_accel_2,years_since_tj,other_pitch_types_X_accel_2,other_pitch_types_Z_accel_2,SO_G_accel_2,ws_pitcher,walk_rate,walk_rate_accel_2,wild_pitch_rate,wild_pitch_rate_accel_2,FA% (pi),FC% (pi),SI% (pi),FA-X (pi),FC-X (pi),SI-X (pi),FA-Z (pi),FC-Z (pi),SI-Z (pi),wFA (pi),wFC (pi),wSI (pi),FA% (pi)_accel_2,FC% (pi)_accel_2,SI% (pi)_accel_2,FA-X (pi)_accel_2,FC-X (pi)_accel_2,SI-X (pi)_accel_2,FA-Z (pi)_accel_2,FC-Z (pi)_accel_2,SI-Z (pi)_accel_2,win_rec,wFA (pi)_accel_2,wSI (pi)_accel_2,wFC (pi)_accel_2,vFA (pi),vFC (pi),vSI (pi),vFC (pi)_accel_2,vFA (pi)_accel_2,vSI (pi)_accel_2,SL% (pi),SL% (pi)_accel_2,SL-X (pi),SL-X (pi)_accel_2,SL-Z (pi),SL-Z (pi)_accel_2,vSL (pi),vSL (pi)_accel_2,wSL (pi),wSL (pi)_accel_2
1275,DAVID PRICE,2008.0,0,0,22.0,236.0,14.0,5.0,NaN,0,0.206,0,16.857143,4.140351,NaN,NaN,NaN,0,NaN,2.800000,47.200000,NaN,NaN,0.0,NaN,NaN,NaN,0,0.070175,NaN,0.000000,NaN,0.691,0.206,NaN,5.1,-1.2,NaN,7.8,2.4,NaN,2.6,0.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,94.8,86.9,NaN,NaN,NaN,NaN,0.296,NaN,-0.7,NaN,2.2,NaN,87.4,NaN,0.7,NaN
1276,DAVID PRICE,2009.0,0,0,23.0,2281.0,128.1,23.0,NaN,0,0.163,0,17.806401,4.095153,NaN,NaN,NaN,0,NaN,5.569565,99.173913,NaN,NaN,0.0,NaN,NaN,NaN,0,0.096948,NaN,0.000877,NaN,0.582,NaN,0.163,5.8,NaN,10.0,9.4,NaN,7.2,5.8,NaN,4.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.588235,NaN,NaN,NaN,94.7,NaN,91.5,NaN,NaN,NaN,0.163,NaN,-0.7,NaN,1.8,NaN,86.3,NaN,-10.7,NaN
1280,DAVID PRICE,2010.0,0,0,24.0,3355.0,208.2,32.0,-0.247610,0,0.197,0,16.114313,3.896632,1039.666667,9.000000,64.733333,0,-0.003000,6.506250,104.843750,-0.017667,0.500000,0.0,-1.566667,-1.533333,1.158333,0,0.090592,0.006806,0.001490,0.000497,0.547,NaN,0.197,6.1,NaN,10.1,9.2,NaN,7.5,13.5,NaN,10.6,-0.048000,NaN,0.065667,0.333333,NaN,3.366667,0.466667,NaN,2.500000,0.760000,3.633333,3.533333,NaN,96.2,NaN,93.5,NaN,0.466667,31.166667,0.050,-0.082,0.1,0.266667,2.6,0.133333,87.2,-0.066667,1.1,0.133333
1283,DAVID PRICE,2011.0,0,0,25.0,3696.0,224.1,34.0,-0.437921,0,0.590,0,16.492637,4.026144,471.666667,3.666667,32.000000,0,0.142333,6.591176,108.705882,-0.018000,5.000000,0.0,0.133333,-0.400000,0.658994,0,0.063181,-0.011256,0.000541,-0.000112,0.208,0.094,0.496,7.0,1.2,10.3,9.3,4.6,7.6,4.6,1.8,10.3,-0.124667,0.031333,0.111000,0.400000,0.400000,0.100000,-0.033333,1.533333,0.133333,0.480000,-0.400000,2.000000,0.600000,95.9,90.0,95.5,30.000000,0.400000,1.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258,DAVID PRICE,2012.0,0,0,26.0,3332.0,211.0,31.0,-0.107615,0,0.644,0,15.791469,3.985646,-7.666667,-0.333333,0.933333,0,0.149000,6.806452,107.483871,-0.008000,3.000000,0.0,0.566667,-0.866667,0.245968,0,0.068182,-0.007470,0.002401,0.000304,0.124,0.159,0.485,7.7,1.1,10.7,9.2,4.5,7.2,7.3,6.3,11.5,-0.141000,0.053000,0.096000,0.533333,0.366667,0.200000,0.000000,1.500000,-0.100000,0.800000,-2.066667,0.300000,2.100000,96.5,89.9,96.2,29.966667,0.100000,0.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1234,DAVID PRICE,2013.0,0,0,27.0,2707.0,186.2,27.0,-0.651502,0,0.572,0,14.538131,3.658108,-329.666667,-2.333333,-12.633333,0,-0.006000,6.896296,100.259259,0.027000,-0.366667,0.0,0.833333,0.200000,-0.273057,0,0.036486,-0.008898,0.002216,0.000558,0.146,0.180,0.392,7.6,1.3,10.8,8.3,4.4,6.5,5.1,0.2,9.3,-0.020667,0.028667,-0.034667,0.200000,0.033333,0.166667,-0.333333,-0.066667,-0.366667,0.555556,0.166667,-0.333333,-0.533333,94.6,89.2,94.5,-0.266667,-0.433333,-0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1245,DAVID PRICE,2014.0,0,0,28.0,3730.0,248.1,34.0,-0.252403,0,0.541,0,15.034260,3.696729,132.666667,1.000000,12.366667,0,-0.034333,7.297059,109.705882,0.018000,-1.866667,0.0,0.500000,0.700000,0.452562,0,0.036670,-0.010504,0.000536,-0.000622,0.173,0.145,0.396,7.7,0.6,

In [ ]:
pitches_injury_2015.groupby("Season").agg({"injury_lagged":"mean"})

In [ ]:
def predict_injury(x,y):
    temp = np.exp(-2.014e+00  + (2.191e-02 * x) + (-6.718e-04 * y)  + (-2.318e-06*x*y))
    return temp / (1.0 + temp)

In [ ]:
np.exp(3)

In [ ]:
predict_injury(7.2, 188)

In [ ]:
predict_injury(610, 35.2)